## Setup

In [1]:
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd()))

from sqlalchemy import create_engine
from utils import get_session
from data.models.eventhub import Ticket

# Import workflow
from agentic.workflow import run_ticket, process_ticket_dict, orchestrator
from agentic.agents.escalation import print_escalation_for_human

# Create database engine
DB_PATH = "data/db/eventhub.db"
engine = create_engine(f"sqlite:///{DB_PATH}")

print("✅ Workflow loaded successfully")

✅ Workflow loaded successfully


## Test 1: Simple General Query (Should Resolve)

In [2]:
# Test with a simple general query
result = run_ticket(
    ticket_id="TKT-GENERAL-001",
    subject="How do I cancel my reservation?",
    description="I need to know the process to cancel a reservation I made for next week.",
    user_email="test@example.com"
)

print("="*80)
print("TEST 1: General Query")
print("="*80)
print(f"\nStatus: {result['final_status'].upper()}")
print(f"Category: {result['classification']['category']}")
print(f"Urgency: {result['classification']['urgency']}")
print(f"Sentiment: {result['classification']['sentiment']}")
print(f"RAG Confidence: {result['rag_confidence']:.1%}")
print(f"\nResponse:")
print("-"*80)
print(result['final_response'])
print("-"*80)

TEST 1: General Query

Status: RESOLVED
Category: cancellation
Urgency: medium
Sentiment: neutral
RAG Confidence: 81.6%

Response:
--------------------------------------------------------------------------------
To cancel your reservation on EventHub, you can follow these steps: 

1. Log in to your account on the EventHub website or app. 
2. Find the event with the reservation you'd like to cancel in your order history. 
3. Click on the 'Cancel' button next to the order. 
4. Confirm the cancellation.

Please note that cancellations are subject to a 24-hour cancellation window and the event's cancellation policy. If you have any questions or issues with your cancellation, you can contact our customer support team at support@eventhub.com. It's also important to check the event's cancellation policy, as it may affect your eligibility for a refund. If you need further assistance, feel free to ask.
--------------------------------------------------------------------------------


## Test 2: Complaint (Should Escalate)

In [3]:
# Test with a complaint (should escalate)
result = run_ticket(
    ticket_id="TKT-COMPLAINT-001",
    subject="Terrible experience - I want my money back!",
    description="""I am extremely frustrated with your service! I booked tickets for the jazz 
    concert last week and when I arrived, the event was CANCELLED without any notification. 
    I drove 2 hours to get there and wasted my entire evening. This is completely unacceptable!
    I demand a full refund AND compensation for my travel expenses.""",
    user_email="angry.customer@email.com",
    reservation_id="RES001"
)

print("="*80)
print("TEST 2: Complaint (Should Escalate)")
print("="*80)
print(f"\nStatus: {result['final_status'].upper()}")
print(f"Category: {result['classification']['category']}")
print(f"Urgency: {result['classification']['urgency']}")
print(f"Sentiment: {result['classification']['sentiment']}")

if result['final_status'] == 'escalated':
    print(f"\n⚠️ ESCALATED")
    print(f"Priority: {result['escalation_details']['priority']}")
    print(f"Reason: {result['escalation_details']['escalation_reason']}")
    print(f"\nCustomer Message:")
    print("-"*80)
    print(result['escalation_details']['customer_message'])
    print("-"*80)

TEST 2: Complaint (Should Escalate)

Status: ESCALATED
Category: refund
Urgency: medium
Sentiment: negative

⚠️ ESCALATED
Priority: 3
Reason: Response indicates need for specialist assistance

Customer Message:
--------------------------------------------------------------------------------
I'm so sorry to hear that you attended a cancelled event without being notified, and I completely understand your frustration. I'm going to connect you with a specialist who will review your situation and work with you to provide a refund and discuss potential compensation. Someone will be with you shortly to help resolve this issue and ensure you receive the assistance you deserve.
--------------------------------------------------------------------------------


## Test 3: Refund Request

In [4]:
# Test with a refund request
result = run_ticket(
    ticket_id="TKT-REFUND-001",
    subject="Requesting refund for cancelled event",
    description="The event I booked was cancelled by the organizer. I would like to request a refund for my tickets.",
    user_email="customer@email.com",
    reservation_id="RES002"
)

print("="*80)
print("TEST 3: Refund Request")
print("="*80)
print(f"\nStatus: {result['final_status'].upper()}")
print(f"Category: {result['classification']['category']}")
print(f"Urgency: {result['classification']['urgency']}")
print(f"Sentiment: {result['classification']['sentiment']}")
print(f"RAG Confidence: {result['rag_confidence']:.1%}")
print(f"\nResponse:")
print("-"*80)
print(result['final_response'])
print("-"*80)

TEST 3: Refund Request

Status: RESOLVED
Category: refund
Urgency: medium
Sentiment: neutral
RAG Confidence: 87.3%

Response:
--------------------------------------------------------------------------------
I'm so sorry to hear that the event you booked was cancelled. If an event is cancelled, you are eligible for a refund. To request a refund, you can log in to your EventHub account on the website or app, navigate to your order history, select the event that was cancelled, and request a refund. Alternatively, you can also contact our support team at support@eventhub.com and we'll be happy to assist you. Please note that refunds will be processed within 7 business days of the cancellation announcement, and you will receive a full refund for your ticket purchase. If you have any questions or concerns, feel free to reach out to us and we'll do our best to help.
--------------------------------------------------------------------------------


## Test 4: Technical Issue

In [5]:
# Test with a technical issue
result = run_ticket(
    ticket_id="TKT-TECH-001",
    subject="App not loading",
    description="The EventHub app keeps crashing when I try to view my reservations. I've tried restarting but it's still not working.",
    user_email="tech.user@email.com"
)

print("="*80)
print("TEST 4: Technical Issue")
print("="*80)
print(f"\nStatus: {result['final_status'].upper()}")
print(f"Category: {result['classification']['category']}")
print(f"Urgency: {result['classification']['urgency']}")
print(f"Sentiment: {result['classification']['sentiment']}")
print(f"RAG Confidence: {result['rag_confidence']:.1%}")
print(f"\nResponse:")
print("-"*80)
print(result['final_response'])
print("-"*80)

TEST 4: Technical Issue

Status: RESOLVED
Category: technical
Urgency: medium
Sentiment: negative
RAG Confidence: 89.1%

Response:
--------------------------------------------------------------------------------
I'm so sorry to hear that the EventHub app is crashing when you try to view your reservations. That can be really frustrating. To help resolve the issue, I recommend trying a few additional steps. First, please make sure you're using the latest version of the app, as updates often include bug fixes and performance improvements. If you're already on the latest version, try clearing the app cache and data. You can do this by going to your device's settings, finding the 'Apps' or 'Application Manager' section, selecting 'EventHub', and tapping 'Clear Cache'. If the problem persists, please contact our support team at support@eventhub.com with your device model and operating system, and we'll do our best to assist you further.
-------------------------------------------------------

## Test 5: Process Real Database Tickets

In [6]:
# Load tickets from database
with get_session(engine) as session:
    db_tickets = session.query(Ticket).limit(5).all()
    
    tickets = [{
        'ticket_id': t.ticket_id,
        'subject': t.subject,
        'description': t.description,
        'user_email': t.user_email,
        'reservation_id': t.reservation_id,
        'actual_category': t.category,
    } for t in db_tickets]

print(f"Loaded {len(tickets)} tickets from database")

Loaded 5 tickets from database


In [7]:
# Process each ticket through the workflow
results = []

for ticket in tickets[:3]:  # Process first 3 to save time
    result = process_ticket_dict(ticket)
    results.append({
        'ticket_id': ticket['ticket_id'],
        'actual_category': ticket['actual_category'],
        'predicted_category': result['classification']['category'],
        'status': result['final_status'],
        'confidence': f"{result['rag_confidence']:.1%}",
    })
    print(f"✅ Processed {ticket['ticket_id']}: {result['final_status']}")

# Display summary
import pandas as pd
df = pd.DataFrame(results)
print("\n" + "="*80)
print("BATCH PROCESSING RESULTS")
print("="*80)
print(df.to_string(index=False))

✅ Processed t_00001: resolved
✅ Processed t_00002: resolved
✅ Processed t_00003: resolved

BATCH PROCESSING RESULTS
ticket_id actual_category predicted_category   status confidence
  t_00001         general            general resolved      71.1%
  t_00002         general            general resolved      79.8%
  t_00003         general            general resolved      75.7%


## View Full Escalation Package (for Human Agent)

In [8]:
# Process a complaint and show full escalation package
result = run_ticket(
    ticket_id="TKT-ESCALATE-DEMO",
    subject="This is unacceptable! Worst service ever!",
    description="I've been waiting for my refund for 3 weeks now and nobody has responded to my emails. I paid $200 for tickets to an event that was cancelled and you're just ignoring me. This is theft!",
    user_email="frustrated@email.com",
    reservation_id="RES003"
)

if result['final_status'] == 'escalated':
    # Build escalation result for display function
    escalation_result = {
        'priority': result['escalation_details']['priority'],
        'customer_message': result['escalation_details']['customer_message'],
        'escalation_package': result['escalation_details']['escalation_package']
    }
    print_escalation_for_human(escalation_result)
else:
    print("Ticket was resolved (not escalated)")

🚨 ESCALATED TICKET - HUMAN AGENT REQUIRED

📊 PRIORITY: 🟠 HIGH
   Reason: Negative sentiment with high urgency requires human attention

────────────────────────────────────────
📋 TICKET INFORMATION
────────────────────────────────────────
  Ticket ID: TKT-ESCALATE-DEMO
  Subject: This is unacceptable! Worst service ever!
  User Email: frustrated@email.com
  Reservation ID: RES003

  Description:
  I've been waiting for my refund for 3 weeks now and nobody has responded to my emails. I paid $200 for tickets to an event that was cancelled and you're just ignoring me. This is theft!

────────────────────────────────────────
🏷️ CLASSIFICATION
────────────────────────────────────────
  Category: REFUND
  Urgency: HIGH
  Sentiment: NEGATIVE
  Summary: Customer awaiting refund for 3 weeks after event cancellation

────────────────────────────────────────
🔧 CONTEXT GATHERED (Tool Results)
────────────────────────────────────────

  📚 KNOWLEDGE BASE ARTICLES:
    1. Can I Get a Refund After the

## Workflow Statistics

In [11]:
# Process more tickets for statistics
with get_session(engine) as session:
    all_tickets = session.query(Ticket).limit(10).all()
    
    # Extract data inside session to avoid DetachedInstanceError
    ticket_data_list = [{
        'ticket_id': t.ticket_id,
        'subject': t.subject,
        'description': t.description,
        'user_email': t.user_email,
        'reservation_id': t.reservation_id,
    } for t in all_tickets]
    
stats = {'resolved': 0, 'escalated': 0, 'categories': {}}

for ticket in ticket_data_list:
    result = run_ticket(
        ticket_id=ticket['ticket_id'],
        subject=ticket['subject'],
        description=ticket['description'],
        user_email=ticket['user_email'],
        reservation_id=ticket['reservation_id']
    )
    
    # Count status
    if result['final_status'] == 'resolved':
        stats['resolved'] += 1
    else:
        stats['escalated'] += 1
    
    # Count categories
    cat = result['classification']['category']
    stats['categories'][cat] = stats['categories'].get(cat, 0) + 1

print("="*80)
print("WORKFLOW STATISTICS")
print("="*80)
print(f"\nTotal Tickets Processed: {len(ticket_data_list)}")
print(f"✅ Resolved: {stats['resolved']} ({stats['resolved']/len(ticket_data_list)*100:.0f}%)")
print(f"⚠️ Escalated: {stats['escalated']} ({stats['escalated']/len(ticket_data_list)*100:.0f}%)")
print(f"\nCategory Distribution:")
for cat, count in sorted(stats['categories'].items(), key=lambda x: -x[1]):
    print(f"  {cat}: {count}")

WORKFLOW STATISTICS

Total Tickets Processed: 10
✅ Resolved: 5 (50%)
⚠️ Escalated: 5 (50%)

Category Distribution:
  general: 10


## Summary

The LangGraph workflow successfully orchestrates:

1. **Classifier Node** → Categorizes tickets (category, urgency, sentiment)
2. **Tool Caller Node** → Fetches KB articles + user/reservation info (CultPass pattern)
3. **Resolver Node** → Generates response using pre-fetched context
4. **Router** → Decides resolved vs escalated (code-based rules)
5. **Response/Escalation Node** → Finalizes output

**Key Features:**
- ✅ Deterministic tool calling (category-based, not LLM-decided)
- ✅ Single LLM call per resolution attempt
- ✅ Code-based escalation rules
- ✅ Full context passed to human on escalation
- ✅ RAG confidence tracking

**Next Steps:**
1. Build Streamlit web app
2. Deploy to EC2